## Importing libraries

In [1]:
import numpy as np
import scipy
from scipy.interpolate import griddata
from scipy.interpolate import Rbf

## Dataset and functions preparation

In [2]:
#Flatten matrix of Cholesky decomposition (lower triangular) into vector
def flatten_matrix(tau):
    flat = np.zeros([tau.shape[0], 4])
    flat[:, 0] = np.real(tau[:, 0, 0])
    flat[:, 1] = np.real(tau[:, 1, 0])
    flat[:, 2] = np.imag(tau[:, 1, 0])
    flat[:, 3] = np.real(tau[:, 1, 1])
    return flat

#Inverse Cholesky transformation using flattened tau
def prediction_to_density(flat):
    tau = np.zeros([flat.shape[0], 2, 2], dtype=np.complex_)
    tau[:, 0, 0] = flat[:, 0]
    tau[:, 0, 1] = 0
    tau[:, 1, 0] = flat[:, 1] + 1j*flat[:, 2]
    tau[:, 1, 1] = flat[:, 3]
    rho = np.zeros([tau.shape[0], 2, 2], dtype=np.complex_)
    for i in range(rho.shape[0]):
        rho[i] = np.dot(tau[i], np.conjugate(np.transpose(tau[i])))
        rho[i] = rho[i] / np.trace(rho[i])
    return rho

#Computing fidelity
def fidelity_array(input_1, input_2):
    fid_int = []
    for i in range(input_1.shape[0]):
        dm_sqrt = scipy.linalg.sqrtm(input_1[i])
        dm_pred = input_2[i]
        in_sqrt = np.linalg.multi_dot([dm_sqrt, dm_pred, dm_sqrt])
        trace = np.trace(scipy.linalg.sqrtm(in_sqrt))
        fidelity = np.real(trace**2)
        fid_int.append(fidelity)
    return np.array(fid_int)

In [3]:
#Loading the dataset of density matrices and control voltages
train_voltage = np.load("Data_files/training_voltages.npy")
val_voltage = np.load("Data_files/validation_voltages.npy")
test_voltage = np.load("Data_files/test_voltages.npy")

train_rho = np.load("Data_files/training_rhos.npy")
val_rho = np.load("Data_files/validation_rhos.npy")
test_rho = np.load("Data_files/test_rhos.npy")

#Flattening the density matrix into vector
train_flat_rho = flatten_matrix(train_rho)
val_flat_rho = flatten_matrix(val_rho)
test_flat_rho = flatten_matrix(test_rho)

#Cholesky decompostion
train_tau = np.linalg.cholesky(train_rho)
val_tau = np.linalg.cholesky(val_rho)
test_tau = np.linalg.cholesky(test_rho)

#Flatting the cholesky decomposition into vector
train_flat_tau = flatten_matrix(train_tau)
val_flat_tau = flatten_matrix(val_tau)
test_flat_tau = flatten_matrix(test_tau)

## Linear interpolation of the direct transformation evaluation

In [4]:
#Linear interpolation of the training set to predict the test set
inter_pred_flat_0 = griddata(train_voltage, train_flat_tau[:,0], 
                             test_voltage, method="linear")
inter_pred_flat_1 = griddata(train_voltage, train_flat_tau[:,1], 
                             test_voltage, method="linear")
inter_pred_flat_2 = griddata(train_voltage, train_flat_tau[:,2], 
                             test_voltage, method="linear")
inter_pred_flat_3 = griddata(train_voltage, train_flat_tau[:,3], 
                             test_voltage, method="linear")

#Handling extrapolation cases by nearest neighbor
nan_pos = np.ndarray.flatten(np.argwhere(np.isnan(inter_pred_flat_0)))
inter_pred_flat_0[nan_pos] = griddata(train_voltage, train_flat_tau[:,0], 
                                      test_voltage[nan_pos], method="nearest")
inter_pred_flat_1[nan_pos] = griddata(train_voltage, train_flat_tau[:,1], 
                                      test_voltage[nan_pos], method="nearest")
inter_pred_flat_2[nan_pos] = griddata(train_voltage, train_flat_tau[:,2], 
                                      test_voltage[nan_pos], method="nearest")
inter_pred_flat_3[nan_pos] = griddata(train_voltage, train_flat_tau[:,3], 
                                      test_voltage[nan_pos], method="nearest")

#Interpolated results
inter_pred_flat = np.transpose(np.array([inter_pred_flat_0, inter_pred_flat_1, 
                                         inter_pred_flat_2, inter_pred_flat_3]))
DLI_test_pred_rho = prediction_to_density(inter_pred_flat)

In [5]:
#Evaluation of results
DLI_fidelities = fidelity_array(test_rho, DLI_test_pred_rho)
print("Average fidelity on the test set:", np.mean(DLI_fidelities))
print("")

DLI_infidelities = 1-DLI_fidelities
print("Average infidelity on the test set:", np.mean(DLI_infidelities))
print("The 5th percentile of infidelities:", np.quantile(DLI_infidelities, 0.05))
print("The 95th percentile of infidelities:", np.quantile(DLI_infidelities, 0.95))
print("")

print("The format of infidelities in the article:")
print("(", np.mean(DLI_infidelities), 
      "-", np.mean(DLI_infidelities)-np.quantile(DLI_infidelities, 0.05), 
      "+", np.quantile(DLI_infidelities, 0.95)-np.mean(DLI_infidelities), ")")

Average fidelity on the test set: 0.9902381737139949

Average infidelity on the test set: 0.009761826286005184
The 5th percentile of infidelities: 6.71462123402744e-08
The 95th percentile of infidelities: 0.017825833574939578

The format of infidelities in the article:
( 0.009761826286005184 - 0.009761759139792844 + 0.008064007288934394 )


## Linear interpolation of the compound transformation evaluation

In [6]:
#Linear interpolation of the inverse part
inter_pred_volt_1 = griddata(train_flat_rho[:,1:], train_voltage[:,0], 
                             test_flat_rho[:,1:], method="linear")
inter_pred_volt_2 = griddata(train_flat_rho[:,1:], train_voltage[:,1], 
                             test_flat_rho[:,1:], method="linear")
inter_pred_volt_3 = griddata(train_flat_rho[:,1:], train_voltage[:,2], 
                             test_flat_rho[:,1:], method="linear")

#Handling extrapolation
nan_pos = np.ndarray.flatten(np.argwhere(np.isnan(inter_pred_volt_1)))
inter_pred_volt_1[nan_pos] = griddata(train_flat_rho[:,1:], train_voltage[:,0], 
                                      test_flat_rho[nan_pos,1:], method="nearest")
inter_pred_volt_2[nan_pos] = griddata(train_flat_rho[:,1:], train_voltage[:,1], 
                                      test_flat_rho[nan_pos,1:], method="nearest")
inter_pred_volt_3[nan_pos] = griddata(train_flat_rho[:,1:], train_voltage[:,2], 
                                      test_flat_rho[nan_pos,1:], method="nearest")

#Interpolated results of the inverse part
inter_pred_volt = np.transpose(np.array([inter_pred_volt_1, inter_pred_volt_2, inter_pred_volt_3]))

In [7]:
#Linear interpolation of the direct part from the inverse part results
inter_pred_flat_0 = griddata(train_voltage, train_flat_tau[:,0], 
                             inter_pred_volt, method="linear")
inter_pred_flat_1 = griddata(train_voltage, train_flat_tau[:,1], 
                             inter_pred_volt, method="linear")
inter_pred_flat_2 = griddata(train_voltage, train_flat_tau[:,2], 
                             inter_pred_volt, method="linear")
inter_pred_flat_3 = griddata(train_voltage, train_flat_tau[:,3], 
                             inter_pred_volt, method="linear")

#Handling extrapolation
nan_pos = np.ndarray.flatten(np.argwhere(np.isnan(inter_pred_flat_0)))
inter_pred_flat_0[nan_pos] = griddata(train_voltage, train_flat_tau[:,0], 
                                      inter_pred_volt[nan_pos], method="nearest")
inter_pred_flat_1[nan_pos] = griddata(train_voltage, train_flat_tau[:,1], 
                                      inter_pred_volt[nan_pos], method="nearest")
inter_pred_flat_2[nan_pos] = griddata(train_voltage, train_flat_tau[:,2], 
                                      inter_pred_volt[nan_pos], method="nearest")
inter_pred_flat_3[nan_pos] = griddata(train_voltage, train_flat_tau[:,3], 
                                      inter_pred_volt[nan_pos], method="nearest")

#Interpolated results of the compound model
inter_pred_flat = np.transpose(np.array([inter_pred_flat_0, inter_pred_flat_1, 
                                         inter_pred_flat_2, inter_pred_flat_3]))
CLI_test_pred_rho = prediction_to_density(inter_pred_flat)

In [8]:
#Evaluation of results
CLI_fidelities = fidelity_array(test_rho, CLI_test_pred_rho)
print("Average fidelity on the test set:", np.mean(CLI_fidelities))
print("")

CLI_infidelities = 1-CLI_fidelities
print("Average infidelity on the test set:", np.mean(CLI_infidelities))
print("The 5th percentile of infidelities:", np.quantile(CLI_infidelities, 0.05))
print("The 95th percentile of infidelities:", np.quantile(CLI_infidelities, 0.95))
print("")

print("The format of infidelities in the article:")
print("(", np.mean(CLI_infidelities), 
      "-", np.mean(CLI_infidelities)-np.quantile(CLI_infidelities, 0.05), 
      "+", np.quantile(CLI_infidelities, 0.95)-np.mean(CLI_infidelities), ")")

Average fidelity on the test set: 0.9901333150506241

Average infidelity on the test set: 0.009866684949376077
The 5th percentile of infidelities: 4.3514608694117395e-09
The 95th percentile of infidelities: 0.021263213365275938

The format of infidelities in the article:
( 0.009866684949376077 - 0.009866680597915208 + 0.011396528415899861 )


## Radial basis funtion interpolation of direct transformation

In [9]:
#Learning the interpolation function using cubic basis set
cubic_interpolation_function = Rbf(train_voltage[:,0], train_voltage[:,1], train_voltage[:,2], 
                                   train_flat_tau, function="cubic", mode="N-D")

#Interpolated results
inter_pred = cubic_interpolation_function(test_voltage[:,0], test_voltage[:,1], test_voltage[:,2])
DRBF_test_pred_rho = prediction_to_density(inter_pred)

In [10]:
#Evaluation of results
DRBF_fidelities = fidelity_array(test_rho, DRBF_test_pred_rho)
print("Average fidelity on the test set:", np.mean(DRBF_fidelities))
print("")

DRBF_infidelities = 1-DRBF_fidelities
print("Average infidelity on the test set:", np.mean(DRBF_infidelities))
print("The 5th percentile of infidelities:", np.quantile(DRBF_infidelities, 0.05))
print("The 95th percentile of infidelities:", np.quantile(DRBF_infidelities, 0.95))
print("")

print("The format of infidelities in the article:")
print("(", np.mean(DRBF_infidelities), 
      "-", np.mean(DRBF_infidelities)-np.quantile(DRBF_infidelities, 0.05), 
      "+", np.quantile(DRBF_infidelities, 0.95)-np.mean(DRBF_infidelities), ")")

Average fidelity on the test set: 0.9984571786528771

Average infidelity on the test set: 0.0015428213471230666
The 5th percentile of infidelities: 3.1901737011508234e-07
The 95th percentile of infidelities: 0.0037912864657654376

The format of infidelities in the article:
( 0.0015428213471230666 - 0.0015425023297529516 + 0.002248465118642371 )


## Radial basis funtion interpolation of compound transformation

In [11]:
#The inverse part of the RBF interpolation 
interpolation_function_inverse = Rbf(train_flat_rho[:,0], train_flat_rho[:,1], train_flat_rho[:,2], 
                                     train_flat_rho[:,3], train_voltage, function="linear", mode="N-D")
inter_pred_inverse = interpolation_function_inverse(test_flat_rho[:,0], test_flat_rho[:,1], 
                                                    test_flat_rho[:,2], test_flat_rho[:,3])

#The direct part of the RBF interpolation using results from the inverse part
interpolation_function_direct = Rbf(train_voltage[:,0], train_voltage[:,1], train_voltage[:,2], 
                                    train_flat_tau, function="cubic", mode="N-D")
inter_pred_direct = interpolation_function_direct(inter_pred_inverse[:,0], inter_pred_inverse[:,1], 
                                                  inter_pred_inverse[:,2])

#Interpolated results
CRBF_test_pred_rho = prediction_to_density(inter_pred_direct)

In [12]:
#Evaluation of results
CRBF_fidelities = fidelity_array(test_rho, CRBF_test_pred_rho)
print("Average fidelity on the test set:", np.mean(CRBF_fidelities))
print("")

CRBF_infidelities = 1-CRBF_fidelities
print("Average infidelity on the test set:", np.mean(CRBF_infidelities))
print("The 5th percentile of infidelities:", np.quantile(CRBF_infidelities, 0.05))
print("The 95th percentile of infidelities:", np.quantile(CRBF_infidelities, 0.95))
print("")

print("The format of infidelities in the article:")
print("(", np.mean(CRBF_infidelities), 
      "-", np.mean(CRBF_infidelities)-np.quantile(CRBF_infidelities, 0.05), 
      "+", np.quantile(CRBF_infidelities, 0.95)-np.mean(CRBF_infidelities), ")")

Average fidelity on the test set: 0.9548109223602367

Average infidelity on the test set: 0.045189077639763306
The 5th percentile of infidelities: 2.5297932119139335e-07
The 95th percentile of infidelities: 0.3255401964717626

The format of infidelities in the article:
( 0.045189077639763306 - 0.04518882466044211 + 0.28035111883199926 )
